# Question 1a: %timeit
You may know from your experiences with matlab that you should always prefer vector- or matrix-based operations over for loops, if possible (hence the name **mat**(rix)**lab**(oratory)). The same is true of python -- you should prefer numpy-array-based operations over for loops. This will also be important for tensorflow -- as much as possible, you should avoid using python for loops when writing tensorflow code. To examine the impact of using for loops over numpy-array-based operations, for this question, you will exploit one of jupyter's built-in magic commands, `%timeit`:

In [1]:
import numpy as np
%timeit np.zeros((100,100))  # provide statistics on how long it takes to generate a 100x100 array of 0s

5.92 µs ± 205 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


As you can see, all you need to do is put `%timeit` before the command that you would normally run and jupyter will run that line multiple times to generate computation timing statistics.

Now, let's compare the computation timing for multiplying two random matrices, each with a dimension of 100x100, using 1) `np.matmul` and 2) multiple embedded for loops. For (2), please write your own function to implement the for loops. Feel free to wrap (2) into a function definition. Verify that (1) and (2) produce the same output. According to `%timeit`, how many times faster is (1) than (2)?

In [18]:
def manualmatmul(a,b):
    mat = []
    for i in range(len(a)):
        vec = []
        for j in range(len(b)):
            
            addme = 0
            for k in range(len(a[i])):
                addme = addme + a[i][k]*b[k][j]
            vec.append(addme)
        mat.append(vec)   
    return mat


In [19]:
a = np.random.rand(100,100)
b = np.random.rand(100,100)

# Use pre-built np.matmul
%timeit np.matmul(a,b)

# The slow way
%timeit manualmatmul(a,b)

149 µs ± 1.5 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
1.26 s ± 13.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The manual method is on the order of 10,000 times faster than `np.matmul`.

# Question 1b
There are two main ways of computing convolutions digitally: 1) directly, using the definition of a convolution, and 2) using the convolution theorem that you proved in the written portion of this homework assignment (i.e., using ffts). Which method is more efficient depends on the sizes of the inputs. Let's use `%timeit` to compare the speeds for 1D convolutions using [`scipy.signal.convolve `](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve.html). This function has an argument called "method", which can be set to "direct" or "fft", which correspond to (1) and (2) above. Use this function to convolve two random 1D signals of lengths $n=100, 500, 1000,$ and $2000$, and compare the speed of both methods. For which n do(es) method 1 outperform method 2, and vice versa? Can you make any generalizations based on these results about when one method outperforms the other?

In [21]:
from scipy.signal import convolve

In [24]:
lengths = [100, 500, 1000, 2000]
for i in lengths:
    a = np.random.rand(i)
    b = np.random.rand(i)
    %timeit convolve(a,b,method='direct')

20.9 µs ± 1.4 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
133 µs ± 1.19 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
386 µs ± 11.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.38 ms ± 30 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [25]:
lengths = [100, 500, 1000, 2000]
for i in lengths:
    a = np.random.rand(i)
    b = np.random.rand(i)
    %timeit convolve(a,b,method='fft')

332 µs ± 11.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
416 µs ± 6.34 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
477 µs ± 2.67 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
621 µs ± 14.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


We see that for small vectors (in this case, less than 1000 in length), the `direct` method is faster than the `fft` method, by up to a factor of about 15 for length=100. However, for larger vectors, the `fft` method is faster by a factor of 2 (for length=2000). Presumably, the transition point of when the `fft` method becomes faster than `direct` is when the time cost of performing the Fourier transform is outweighed by the cost of performing individual shifts/multiplications by the definition of a convolution. For larger vectors, because the time cost of performing the Fourier transform is small relative to performing matrix multiplcation, the `fft` is essentially only limited by the speed of matrix multiplication.

# Question 2: the convolution theorem
As we investigated in question 1b, it is also possible to do convolutions using Fourier transforms, and in some cases this is the preferable method. In fact, there is some body of work investigating the use of ffts and multiplication to do convolution operations in convolutional neural networks. 

For this question, to illustrate this theorem, given a convolutional kernel you will find the corresponding Fourier operation that produces the same result. To this end,

1. create a 7x7 Gaussian kernel with a standard deviation $\sigma=2$ (using a pixel grid spacing of 1)
2. load an image, if it is color then convert it to grayscale (you can just sum the 3 color channels), and then resize the image into a 128x128 array
3. compute the convolution  - you can use a numpy (np) or scipy function. Make sure the output is the same size as the input image, which is slightly different than the formal definition of a discrete convolution, but is something that is usually convenient to do.
4. Find the Fourier filter that does the same operation in the Fourier domain, and show the resulting blurred image implemented using the Fourier method (i.e., if $I_{2}=I_{1}*h$, then $\mathcal{F}[I_{2}]=\mathcal{F}[I_{1}]\mathcal{F}[h]$, so find the correct array for $\mathcal{F}[h]$ and re-generate $I_2$). 

In [ ]:
# the following line will cause subsequent plotting commands to display directly in the notebook
%matplotlib inline

# Question 3: data augmentation
One indispensable tool used in deep learning is data augmentation. That is, we can to some extent artificially increase the size of our dataset by randomly altering the current dataset. One common augmenting operation is to do random crops of the original image. For example, researchers designing neural networks for ImageNet, a dataset of natural RGB images, typically resize the images to 256x256x3 and then take a random 224x224x3 crop such that the latter fits entirely in the former.

For this question, take a picture with your phone or find a picture online, load it into jupyter, resize it to 256x256x3 (discard the alpha channel if one is present), and then perform the random 224x224x3 crop. The crops should be uniformly distributed within the bounding 256x256 box and do not need to be rotated. Please display the 256x256x3 image and 5 random crops using `plt.imshow`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# your code here; feel free to use multiple cells